In [1]:
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import datetime
import shutil
from six.moves import urllib
from pathlib import Path
import trackpy
from iris.time import PartialDateTime

import tobac # #tobac package cloned from https://github.com/tobac-project/tobac.git

import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [2]:
print(tobac.__version__)

1.5.0


In [3]:
#load in file
data_out=('../')
data_file = '/data/users/hgilmour/tb/2005/tb_merge_01_2005.nc' #this is the 1 hourly mean olr data for the first 20 days of 1998
print(data_file)

tb=iris.load_cube(data_file)


#constraining the dataset by time so it runs quicker: 
#week=iris.Constraint(time=lambda cell: cell.point.day<=7) #just the first 7 days
#olr = olr.extract(week)

# olr.coord('time').bounds=None #REMOVING BOUNDS FROM TIME TO SEE IF THIS HELPS THE TYPEERROR
# # Remove coord system or else the animations don't run (suggested by AVD team)
# olr.coord('latitude').coord_system = None
# olr.coord('longitude').coord_system = None

# #code that AVD team sent:
# time = olr.coord('time')
# datetimes = time.units.num2date(time.points)
# con = iris.Constraint(time=datetimes[0])
# olr.extract(con)


# print(olr)

#Set up directory to save output and plots:
savedir=Path("Save")
if not savedir.is_dir():
    savedir.mkdir()
plot_dir=Path("Plot")
if not plot_dir.is_dir():
    plot_dir.mkdir()      


#calculating dxy
from math import pi
longitude,latitude=np.meshgrid(tb.coord('longitude').points,tb.coord('latitude').points)
R=6.3781e6
dx=np.gradient(longitude)[1]
dx=dx*(pi/180)*R*np.cos(latitude*pi/180)
dy=np.gradient(latitude)[0]
dy=dy*(pi/180)*R
print(dx)
print(dy)


# Determine temporal and spatial sampling of the input data:
dxy,dt=tobac.get_spacings(tb,grid_spacing=4500,time_spacing=3600) #time spacing = 1 hour

/data/users/hgilmour/tb/2005/tb_merge_01_2005.nc
[[3451.3477 3451.3477 3451.3477 ... 3451.3477 3451.3477 3451.3477]
 [3453.3967 3453.3967 3453.3967 ... 3453.3967 3453.3967 3453.3967]
 [3455.4443 3455.4443 3455.4443 ... 3455.4443 3455.4443 3455.4443]
 ...
 [4356.1074 4356.1074 4356.1074 ... 4356.1074 4356.1074 4356.1074]
 [4355.286  4355.286  4355.286  ... 4355.286  4355.286  4355.286 ]
 [4354.4624 4354.4624 4354.4624 ... 4354.4624 4354.4624 4354.4624]]
[[4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.272  4508.272  4508.272  ... 4508.272  4508.272  4508.272 ]
 ...
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]]


In [4]:
# dxy,dt=tobac.get_spacings(olr,grid_spacing=4500,time_spacing=3600) #time spacing = 1 hour


# #Convert from olr to Tb using Ohring and Gruber empirical formula:
# # # (1984) as given in Yang and Slingo (2001)
#     # Tf = tb(a+b*Tb) where a = 1.228 and b = -1.106e-3 K^-1
#     # OLR = sigma*Tf^4 
#     # where sigma = Stefan-Boltzmann constant = 5.67x10^-8 W m^-2 K^-4
# a = 1.228
# b = -1.106e-3
# sigma = 5.67e-8 # W m^-2 K^-4

# tf = (olr.data/5.67e-8)**0.25
# tb_var = (-1.228 + np.sqrt(1.228**2 + 4*-1.106e-3*tf.data))/(2*-1.106e-3)

# tb=olr.copy()
# tb.data=tb_var.data
# print(tb)

In [5]:

#Parameter features:
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['target']='minimum'
parameters_features['threshold']=240 #olr threshold equivalent to Tb=225K based on stefan boltzmann equation (145 for 225K, 91 for 200K, 74 for 190K)
parameters_features['n_min_threshold']=1975 # number of grid points for 40,000km^2 area (7792m = 1 grid space. 4500m x 4500m = 20250000m^2. 40,000km^2 = 4x10^10m^2. 4x10^10 / 20250000 = 1975 (1975 grid cells per 40,000km^2 area)

# Feature detection and save results to file:
print('starting feature detection')
Features=tobac.feature_detection_multithreshold(tb,dxy,**parameters_features)
Features.to_hdf('Save/features_2005_01.h5','table')
print('feature detection performed and saved')

starting feature detection
feature detection performed and saved


In [6]:
# Segmentation:
parameters_segmentation={}
parameters_segmentation['target']='minimum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=240


# Perform segmentation and save results to files:
Mask_tb,Features_tb=tobac.segmentation_2D(Features,tb,dxy,**parameters_segmentation)
print('segmentation tb performed, start saving results to files')
iris.save([Mask_tb], 'Save/mask_2005_01.nc', zlib=True, complevel=4)
Features_tb.to_hdf('Save/features_tb_2005_01.h5', 'table')
print('segmentation tb performed and saved')

segmentation tb performed, start saving results to files
segmentation tb performed and saved


In [7]:
# Linking:
parameters_linking={}
parameters_linking['v_max']=60 #(velocity of 60 m s-1 is referenced in https://journals.ametsoc.org/view/journals/mwre/126/6/1520-0493_1998_126_1630_lcvomc_2.0.co_2.xml#i1520-0493-126-6-1630-f01 study)
parameters_linking['stubs']=7 #minimum number of timesteps for a tracked cell to be reported (equivalent to 6 hours)
parameters_linking['order']=1
parameters_linking['extrapolate']=0 
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=15
parameters_linking['method_linking']= 'predict'



# Perform linking and save results to file:
Track=tobac.linking_trackpy(Features,tb,dt=dt,dxy=dxy,**parameters_linking)
Track["longitude"]=Track["longitude"]-360
Track.to_hdf('Save/tracks_2005_01.h5','table')


Frame 743: 18 trajectories present.


In [8]:
df = pd.read_hdf('Save/tracks_2005_01.h5','table')

In [11]:
df

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,forecast_reference_time,forecast_period,cell,time_cell
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,-64.032043,295.967957,295.967957,1,0 days 00:00:00
1,0,26,319.360864,876.726404,7359,240,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-27.105936,-49.532534,310.467466,310.467466,2,0 days 00:00:00
2,0,57,465.549812,1014.466367,5384,240,3,2005-01-01 00:30:00,2005-01-01 00:30:00,-21.185283,-43.954061,316.045939,316.045939,-1,NaT
3,0,73,763.319602,705.760312,47759,240,4,2005-01-01 00:30:00,2005-01-01 00:30:00,-9.125606,-56.456667,303.543333,303.543333,-1,NaT
4,0,86,577.470368,717.240701,2132,240,5,2005-01-01 00:30:00,2005-01-01 00:30:00,-16.652501,-55.991687,304.008313,304.008313,-1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9076,743,241,882.032521,698.342702,7910,240,9077,2005-01-31 23:30:00,2005-01-31 23:30:00,-4.317733,-56.757057,303.242943,303.242943,-1,NaT
9077,743,252,940.225439,833.293263,3228,240,9078,2005-01-31 23:30:00,2005-01-31 23:30:00,-1.960920,-51.291560,308.708440,308.708440,-1,NaT
9078,743,272,1046.778633,1275.959222,20583,240,9079,2005-01-31 23:30:00,2005-01-31 23:30:00,2.354484,-33.363590,326.636410,326.636410,-1,NaT
9079,743,320,1097.697734,916.296729,3373,240,9080,2005-01-31 23:30:00,2005-01-31 23:30:00,4.416709,-47.929939,312.070061,312.070061,-1,NaT


In [12]:
df = df[df.cell >= 0]

In [13]:
print(np.unique(df.cell.values).shape[0])

391
